## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-03-14-50-02 +0000,nypost,"Georgia releases Bella Culley, pregnant UK tee...",https://nypost.com/2025/11/03/world-news/georg...
1,2025-11-03-14-48-00 +0000,wsj,Dodgers-Blue Jays World Series Game 7 Draws Mo...,https://www.wsj.com/business/media/dodgers-blu...
2,2025-11-03-14-45-56 +0000,nypost,‘60 Minutes’ edits out Trump’s bragging about ...,https://nypost.com/2025/11/03/media/60-minutes...
3,2025-11-03-14-42-00 +0000,wsj,Kimberly-Clark Strikes $40 Billion Deal for Ty...,https://www.wsj.com/business/retail/kimberly-c...
4,2025-11-03-14-38-58 +0000,nyt,Louvre Heist: What to Know About the Suspects ...,https://www.nytimes.com/2025/11/03/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
22,trump,31
47,will,12
132,new,11
199,against,10
51,mamdani,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
152,2025-11-03-02-20-39 +0000,nypost,"Trump talks NYC mayoral race, government shutd...",https://nypost.com/2025/11/02/us-news/trump-ta...,77
204,2025-11-02-20-00-00 +0000,wsj,The Trump administration believes it can conti...,https://www.wsj.com/politics/national-security...,74
99,2025-11-03-09-49-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...,73
60,2025-11-03-11-33-05 +0000,startribune,Government shutdown could become longest ever ...,https://www.startribune.com/government-shutdow...,63
158,2025-11-03-01-32-00 +0000,wsj,Democratic senators again urged President Trum...,https://www.wsj.com/politics/policy/democrats-...,62


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
152,77,2025-11-03-02-20-39 +0000,nypost,"Trump talks NYC mayoral race, government shutd...",https://nypost.com/2025/11/02/us-news/trump-ta...
6,43,2025-11-03-14-32-42 +0000,nypost,Linda Sarsour vows she will ‘hold Zohran accou...,https://nypost.com/2025/11/03/us-news/linda-sa...
204,43,2025-11-02-20-00-00 +0000,wsj,The Trump administration believes it can conti...,https://www.wsj.com/politics/national-security...
230,35,2025-11-02-16-11-31 +0000,bbc,"'No help, no food, no water': Hurricane-hit Ja...",https://www.bbc.com/news/articles/cpv11w8ez10o...
30,34,2025-11-03-13-25-51 +0000,nypost,"Lone survivor of Air India crash speaks out, r...",https://nypost.com/2025/11/03/world-news/lone-...
47,34,2025-11-03-12-05-39 +0000,nypost,Sex drugs found after ex-Prince Andrew threw p...,https://nypost.com/2025/11/03/world-news/sex-d...
59,34,2025-11-03-11-47-41 +0000,nypost,Mississippi mom guns down monkey that escaped ...,https://nypost.com/2025/11/03/us-news/mississi...
125,34,2025-11-03-07-14-02 +0000,bbc,China to ease chip export ban in new trade dea...,https://www.bbc.com/news/articles/cy5qgy93w9go...
161,30,2025-11-03-01-13-26 +0000,bbc,Trump tariffs head to Supreme Court in case ea...,https://www.bbc.com/news/articles/cn4jyk9jyv3o...
164,28,2025-11-03-00-30-49 +0000,nypost,Mexican mayor who stood up to cartels is gunne...,https://nypost.com/2025/11/02/world-news/mexic...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
